In [ ]:
! pip install -q -U unidecode PyPDF2 beautifulsoup4 regex datasets

In [ ]:
import unidecode, random, string, time, pickle, requests, json, csv
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from collections import OrderedDict
from PyPDF2 import PdfReader
import regex as re
import torch
from datasets import load_dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## 1.Gather Text

In [ ]:
# dataset = load_dataset("JeanKaddour/minipile",split='train',trust_remote_code=True)

In [ ]:
dataset2 = load_dataset('dvilasuero/distillama3-prompts10k', split='train')
dataset3 = load_dataset("tatsu-lab/alpaca", split = "train")
dataset4 = load_dataset("garage-bAInd/Open-Platypus", split = "train")

In [ ]:
# dataset5 = load_dataset("tatsu-lab/alpaca", split = "train")
# dataset6 = load_dataset("THUDM/webglm-qa", split = "train")

In [ ]:
len(dataset2),len(dataset3),len(dataset4)

In [ ]:
def load_dataset_(dataset):
    idx = random.randint(0,len(dataset)-1)
    text = f"Instruction:\n {dataset['instruction']} \nInput:\n{dataset['input']} \nOutput:\n{dataset['instruction']}"
    return text

In [ ]:
text = load_dataset_(dataset3)

## 2.Training Tokenizer

In [ ]:
def load_vocab(path):
    f =  open(path)
    vocab = json.load(f)
    f.close()
    vocab = [i.encode('utf-8') for i in vocab]
    return vocab

def load_merges(path):
    with open(path,'rb') as f:
        merges = pickle.load(f)
    return merges

vocab = load_vocab(r'/content/drive/MyDrive/Pretrained_Models/vocab.json')
merges = load_merges(r'/content/drive/MyDrive/Pretrained_Models/merges.pkl')

### 2.1 Byte pair Encoding

Resource: https://github.com/karpathy/minbpe

In [ ]:
class BasicTokenizer():
    def __init__(self, vocab=None, merges=None) -> None:
        self.pattern = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        self.compiled_pattern = re.compile(self.pattern)
        self.vocab = vocab if vocab is not None else {}
        self.merges = merges if merges is not None else {}

    def get_stats(self, ids, counts=None):
        counts = {} if counts is None else counts
        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1
        return counts

    def merge(self, ids, pair, idx):
        newids = []
        i = 0
        while i < len(ids):
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

    def train(self, text, vocab_size, verbose, merges_dir,  vocab_dir):
        text = re.sub(r'[^\w\p{P}\p{M}\p{Z}}"]+',' ',text)
        num_merges = vocab_size - 256
        text_chunks = re.findall(self.compiled_pattern, text)
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        for i in tqdm(range(num_merges),desc='Token merges'):
            stats = {}
            for chunk_ids in ids:
                stats = self.get_stats(chunk_ids, stats)
            if len(stats) > 0:
                pair = max(stats, key=lambda x: stats[x])
                idx = 256 + i
                ids = [self.merge(chunk_ids, pair, idx) for chunk_ids in ids]
                self.merges[pair] = idx
                self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]
                if verbose:
                    print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({self.vocab[idx]}) had {stats[pair]} occurrences")
            else:
                pass
        self.save_merges(merges_dir)
        self.save_vocab(vocab_dir)
        return self.merges, self.vocab

    def encode_chunk(self,text_bytes):
            # return the token ids
            # let's begin. first, convert all bytes to integers in range 0..255
            # text_bytes = bytes(text.encode('utf-8'))
            ids = list(text_bytes)
            while len(ids) >= 2:
                # find the pair with the lowest merge index
                stats = self.get_stats(ids)
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
                # subtle: if there are no more merges available, the key will
                # result in an inf for every single pair, and the min will be
                # just the first pair in the list, arbitrarily
                # we can detect this terminating case by a membership check
                if pair not in self.merges:
                    break # nothing else can be merged anymore
                # otherwise let's merge the best pair (lowest merge index)
                idx = self.merges[pair]
                ids = self.merge(ids, pair, idx)
            return ids

    def decode(self, ids):
        text_bytes = b"".join([self.vocab[i] for i in ids])
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def encode_ordinary(self, text):
        """Encoding that ignores any special tokens."""
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8") # raw bytes
            chunk_ids = self.encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids

    def save_vocab(self, save_path):
        data = [i.decode('utf-8',errors='replace') for i in list(self.vocab.values())]
        with open(save_path,'w') as file:
            json.dump(data,file,indent=4)

    def save_merges(self, save_path):
        with open(save_path,'wb') as f:
            pickle.dump(self.merges,f)

def load_vocab(path):
    f =  open(path)
    vocab = json.load(f)
    f.close()
    vocab = [i.encode('utf-8') for i in vocab]
    return vocab

def load_merges(path):
    with open(path,'rb') as f:
        merges = pickle.load(f)
    return merges


In [ ]:
#tokenizer = BasicTokenizer(vocab,merges)

In [ ]:
# tokenizer.train(text3, 900, False)

`Note`: Byte Pair encoding is not used to fit the training within the T4's memory. Use as small as vocabulary as possible. For this notebook we use single character printable in python as vocabulary. (As you might see later this is also hard to fit)

In [ ]:
vocab = list(string.printable)
print(len(vocab))

### 2.2 Character Level Tokenizer

In [ ]:
class CharacterTokenizer:
    def __init__(self, vocab = None) -> None:
        self.vocab = vocab if vocab is not None else list(string.printable)
        self.vocab = list(string.printable)
        self.stoi = {s:i for i,s in enumerate(self.vocab)}
        self.itos = {i:s for i,s in enumerate(self.vocab)}

    def encode_text(self,text):
        encoded = [self.stoi[words] for words in text]
        return encoded

    def decode_ids(self,ids):
        decoded = ''.join([self.itos[idx] for idx in ids])
        return decoded

## 3.Getting Dataloaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
class Sampler:
    def __init__(self,tokenizer=None):
        self.tokenizer = tokenizer if tokenizer is not None else CharacterTokenizer

    def random_portion(self, text, chunk_size):
        start_idx = random.randint(0,len(text) - chunk_size)
        end_idx = start_idx + chunk_size + 1
        text = text[start_idx: end_idx]
        return text

    def read_pdf(self,path):
        text = ""
        pdf_reader = PdfReader(path)
        index = random.randint(0,len(pdf_reader.pages)-1)
        for page in pdf_reader.pages[index:index+1]:
            text += page.extract_text()
        text = unidecode.unidecode(text)
        return text

    def preprocess(self,text):
        text = unidecode.unidecode(text)
        text = re.sub(r'[^\p{L}\p{N}\p{M}\p{Z}]+',' ',text)
        return text

    def generate_context_target_words(self, ids,seq_len):
        context, targets = [], []
        for i in range(seq_len,len(ids)-seq_len):
            context.append(ids[i-seq_len:i])
            targets.append(ids[i-seq_len+1:i+1])
        return context, targets

    def get_dataloaders(self,context,targets,batch_size,device):
        X = torch.tensor(context,device=device)
        y = torch.tensor(targets,device=device)
        dataset = TensorDataset(X,y)
        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        train_dataset, valid_dataset = random_split(dataset, [train_size, test_size])

        train_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=batch_size)
        valid_loader = DataLoader(dataset=valid_dataset,shuffle=True,batch_size=batch_size)
        return train_loader, valid_loader


    def __call__(self, seq_len, batch_size, device,  text=None, pdf_path=None, chunk_size=1000):
        if pdf_path is not None:
            text = self.read_pdf(pdf_path)
        else:
            pass
        text = self.random_portion(text, chunk_size)
        text = self.preprocess(text)
        ids = self.tokenizer.encode_text(text)
        context, targets = self.generate_context_target_words(ids, seq_len)
        train_loader, valid_loader = self.get_dataloaders(context, targets, batch_size, device)
        return train_loader, valid_loader

## 4.Loading Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.profiler import profile, record_function, ProfilerActivity
from dataclasses import dataclass

In [ ]:
tokenizer = CharacterTokenizer(vocab)
sampler = Sampler(tokenizer)

### 4.1 Model Configuration

In [ ]:
@dataclass
class ModelArgs:
    in_ = len(vocab)
    batch = 128
    seq_len = 128
    dim = 128
    q_heads = 8
    kv_heads = 2
    hdim = dim // q_heads
    blocks = 4
    dropout = 0.2

### 4.2 Root Mean Square Normalisation

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, n_dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(n_dim))

    def forward(self, x):
        x_normed = (
            x.float() * torch.rsqrt(x.float().pow(2).mean(-1, keepdim=True) + self.eps)
        ).type_as(x)
        return x_normed * self.scale

### 4.3 SwiGLU (Activation Function)

In [ ]:
class SwiGLU(nn.Module):
    # SwiGLU(x,W,V,b,c) = x.sigmoid(BxW + b) * (xV+c), *-> element wise product
    def __init__(self,n_dim):
        super().__init__()
        self.linear_gate= nn.Linear(n_dim, n_dim)
        self.linear = nn.Linear(n_dim,n_dim)
        self.beta = nn.Parameter(torch.ones(1))
        self.register_parameter('beta', self.beta)

    def forward(self,x):
        swish_gate = x * torch.sigmoid(self.beta * self.linear_gate(x))
        out = swish_gate * self.linear(x)
        return out

### 4.4 Rotary Positional Embeddings

In [ ]:
def get_rotary_matrix(seq_len, n_dim, base=10000):
    R = torch.zeros((seq_len, n_dim, n_dim),requires_grad=False)
    for p in range(seq_len):
        for i in range(n_dim // 2):
            theta = base ** (-2. * (i-1)/n_dim)
            m_theta = torch.tensor(p * theta)
            R[p, 2*i, 2*i] = torch.cos(m_theta)
            R[p, 2*i, 2*i+1] = -torch.sin(m_theta)
            R[p, 2*i+1, 2*i] = torch.sin(m_theta)
            R[p, 2*i+1, 2*i+1] = torch.cos(m_theta)
    return R

### 4.5 RoPE Self Attention

In [ ]:
class RoPEMaskedAttentionHead(nn.Module):
    def __init__(self, args:ModelArgs):
        super().__init__()
        self.wq = nn.Linear(args.dim,args.dim, bias=False)
        self.wk = nn.Linear(args.dim,args.dim, bias=False)
        self.wv = nn.Linear(args.dim,args.dim, bias=False)
        self.R = get_rotary_matrix(args.seq_len,args.dim)
        self.drop = args.dropout

    def forward(self, x):
        q = self.wq(x)
        k = self.wk(x)
        v = self.wv(x)
        self.R = self.R.to(x.device)
        q_rotated = (torch.bmm(q.transpose(0, 1), self.R[:x.shape[1]])).transpose(0, 1)
        k_rotated = (torch.bmm(k.transpose(0, 1), self.R[:x.shape[1]])).transpose(0, 1)
        activations = F.scaled_dot_product_attention(q_rotated,k_rotated,v,dropout_p=self.drop,is_causal=True)
        return activations

### 4.6 RoPE Multi-Head Attention

In [ ]:
class RoPEMaskedMultiheadAttention(nn.Module):
    def __init__(self, args:ModelArgs):
        super().__init__()
        self.heads = nn.ModuleList([
            RoPEMaskedAttentionHead(args) for _ in range(args.q_heads)
        ])
        self.linear = nn.Linear(args.q_heads*args.dim, args.dim)
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):
        heads = [h(x) for h in self.heads]
        x = torch.cat(heads, dim=-1)
        x = self.linear(x)
        x = self.dropout(x)
        return x

### 4.7 Decoder Model Block

In [ ]:
class ModelBlock(nn.Module):
    def __init__(self, args:ModelArgs):
        super().__init__()
        self.rms = RMSNorm(args.dim)
        self.rope_attention = RoPEMaskedMultiheadAttention(args)
        self.linear = nn.Sequential(
            nn.Linear(args.dim, args.dim),
            SwiGLU(args.dim)
        )

    def forward(self, x):
        x = self.rms(x)
        x = x + self.rope_attention(x)
        x = self.rms(x)
        x = x + self.linear(x)
        return x

### 4.8 Complete Model

In [ ]:
class Model(nn.Module):
    def __init__(self, args:ModelArgs):
        super().__init__()
        self.embeddings = nn.Embedding(args.in_,args.dim)
        self.model_blocks = nn.Sequential(
            OrderedDict([(f"block_{i}", ModelBlock(args)) for i in range(args.blocks)])
        )
        self.ffn = nn.Sequential(
            nn.Linear(args.dim,args.dim),
            SwiGLU(args.dim),
            nn.Linear(args.dim,args.in_)
        )
    def forward(self, idx):
        x = self.embeddings(idx)
        x = self.model_blocks(x)
        logits = self.ffn(x)
        return logits

## 5.Setup Model

### 5.1 Training Configuration

In [ ]:
@dataclass
class TrainingArgs:
    epochs = 1e3
    lr = 1e-3
    scheduler_gamma = 0.9999
    use_mixed_precision = True
    model_save_dir = r'/content/drive/MyDrive/Pretrained_Models/model_checkpoint_4.pt'
    model_load_dir = model_save_dir
    loss_dir = r'/content/drive/MyDrive/Pretrained_Models/loss.csv'

In [ ]:
def save_checkpoint(model, optimizer, save_path:str, epoch:int, scaler):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler':scaler.state_dict(),
        'epoch': epoch,
    }, save_path)

def load_checkpoint(model, optimizer, scaler, load_path:str,device):
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler'])
    epoch = checkpoint['epoch']
    return model, optimizer, scaler, epoch

In [ ]:
def save_loss(loss_data,save_dir):
    with open(save_dir, 'w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(loss_data)

In [ ]:
def train(model, args:ModelArgs, text, train_args:TrainingArgs, dataloader, optimizer, scheduler, device, checkpointing=False, save_loss=False):
    # Gradient scaling helps prevent gradients with small magnitudes from flushing to zero (“underflowing”) when training with mixed precision.
    scaler = torch.cuda.amp.GradScaler()
    loss_logs = []
    if checkpointing is True:
        model, optimizer, scaler, epoch = load_checkpoint(model,optimizer,scaler, train_args.model_load_dir, device)
    try:
        model.train()
        start = time.time()
        for epoch in tqdm(range(int(train_args.epochs)),desc='Epochs',total=train_args.epochs,maxinterval=train_args.epochs//10):
            train_dataloader, validation_dataloader = dataloader(args.seq_len, args.batch, device, text)
            for x_batch, y_batch in train_dataloader:
                with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=train_args.use_mixed_precision):
                    y_pred = model(x_batch)
                    train_loss = F.cross_entropy(y_pred.view(-1,y_pred.shape[2]),y_batch.view(-1))
                scaler.scale(train_loss).backward()
                # nn.utils.clip_grad_norm_(model.parameters(),1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

            model.eval()
            with torch.inference_mode():
                for x_batch, y_batch in validation_dataloader:
                    y_pred = model(x_batch)
                    validation_loss = F.cross_entropy(y_pred.view(-1,y_pred.shape[2]),y_batch.view(-1))

            if epoch %  (train_args.epochs/10) == 0:
                batch_time = time.time() - start
                model.eval()
                gpu_usage = round(torch.cuda.memory_reserved(0)/1024**3,1)
                print(f'\nEpoch: {epoch} | Train Loss : {train_loss.detach().item():.4f} | Eval Loss : {validation_loss.detach().item():.4f} | Time : {batch_time:.2f} s | Learning Rate: {scheduler.get_last_lr()[0]:.2e} | Gpu Usage: {gpu_usage}\n')
                model.train()
            scheduler.step()
            loss_logs.append((train_loss.detach().item(), validation_loss.detach().item()))
    except KeyboardInterrupt:
        save_checkpoint(model,optimizer,train_args.model_save_dir,epoch, scaler)

    if TrainingArgs.model_save_dir is not None:
        save_checkpoint(model,optimizer,train_args.model_save_dir,epoch, scaler)

    if save_loss is True:
        save_loss(loss_logs,TrainingArgs.loss_dir)

In [ ]:
def diagnose_model(model, args:ModelArgs, device=None):
    x = torch.randint(0,100,(args.batch,args.seq_len),device=device)
    with profile(activities=[
            ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
            model(x)

    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
    print('Number of Parameters in the Model:',sum(param.numel() for param in model.parameters()))

In [ ]:
model = Model(ModelArgs)
model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),TrainingArgs.lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9999)

## 6.Run Diagnostics

In [ ]:
diagnose_model(model,ModelArgs,device)

## 7.Train Model

In [ ]:
train(
    model=model,
    args=ModelArgs,
    text=text,
    train_args=TrainingArgs,
    dataloader=sampler,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    checkpointing=True, # False on first Run
    save_loss=True,
    )

## 8.Inference

### 8.1 Greedy Decoding

In [ ]:
def model_generate_greedy(model, input_ids, seq_len, gen_len, temperature):
    model.eval()
    with torch.inference_mode():
        for _ in range(gen_len):
            X_trunc = input_ids[:,-seq_len:] # truncate all besides last context length terms
            logits = model(X_trunc)
            logits = logits / temperature # Scales the logits. Lower temperature makes higher logits peaky leading to higher confidence
            logits = logits[:,-1,:] # use only last token for next token generation
            probs = F.softmax(logits,dim=-1)
            next_tok = torch.multinomial(probs,num_samples=1)
            input_ids = torch.cat((input_ids,next_tok),dim=1)
    #gen = tokenizer.decode_ids(x[0].tolist())
    return input_ids

### 8.2 Top-k Sampling

In [ ]:
def top_k_sampling(input_ids, model, max_tokens=100, top_k=50, temperature=1.0):
    for _ in range(max_tokens):
        # Temporarily disables gradient calculation to improve performance and reduce memory usage
        with torch.inference_mode():
            logits = model(input_ids)
            logits = logits[:, -1, :]
            # Select the top K tokens from the probability distribution
            top_k_logits, top_k_indices = torch.topk(logits, top_k)

            # Apply softmax to convert logits to probabilities, with optional temperature scaling
            top_k_probs = F.softmax(top_k_logits / temperature, dim=-1)

            # Sample from the top K tokens to determine the next token
            next_token_index = torch.multinomial(top_k_probs, num_samples=1)

            # Map the sampled token back to its original index in the logits tensor
            next_token = top_k_indices.gather(-1, next_token_index)

            # Concatenate the new token to the sequence of input IDs
            input_ids = torch.cat([input_ids, next_token], dim=-1)
    return input_ids[0].tolist()

### 8.3 Top-p Sampling

In [ ]:
def top_p_sampling(input_ids: list[int], model, temperature: float=1, max_tokens :int=100, top_p: float=0.95):
    with torch.inference_mode():
        for _ in range(max_tokens):
            logits = model(input_ids)
            logits = logits[:, -1, :]

            # Sort the logits in descending order and apply softmax to get probabilities
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            sorted_probabilities = F.softmax(sorted_logits / temperature, dim=-1) # Apply temperature

            # Calculate cumulative probabilities
            cumulative_probs = torch.cumsum(sorted_probabilities, dim=-1)

            # Identify and remove tokens with cumulative probability above the threshold (top_p)
            # Ensuring the first token is always selected by setting the first position to False
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 0] = False  # Keep the most probable token

            # Get the actual indices to remove from the original logits tensor
            indices_to_remove = sorted_indices[sorted_indices_to_remove]

            # Set the logits of removed tokens to -infinity to exclude them from sampling
            logits.scatter_(-1, indices_to_remove[None, :], float('-inf'))

            # Re-calculate probabilities after filtering and sample from this distribution
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

            # Append the sampled token to the input IDs for the next iteration
            input_ids = torch.cat([input_ids, next_token], dim=-1)

    return input_ids[0].tolist()

In [ ]:
scaler = torch.cuda.amp.GradScaler()
model, _, _, _ =load_checkpoint(model,optimizer,scaler,TrainingArgs.model_load_dir,device)

In [ ]:
prompt = 'What'
input_ids = torch.tensor(tokenizer.encode_text(prompt),device=device).view(1,-1)
decoded_ids = top_p_sampling(input_ids, model)
print(tokenizer.decode_ids(decoded_ids))

## 9. Plots

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

In [ ]:
def plot_(path):
    df = pd.read_csv(path).dropna()
    train = df['train'].tolist()
    val = df['validation'].tolist()
    plt.plot(range(len(train)),train)
    plt.plot(range(len(val)),val)
    plt.xlabel('epochs')
    plt.ylabel('cross-entropy')
    plt.title('Loss Curves')
    plt.legend(['train_loss','validation_loss'])
    plt.show()